# Stack Overflow Question Generation using LLMs

This notebook combines processing using different LLMs:
- Google's Gemini
- Meta's LLaMA
- OpenAI's GPT

Each section implements different prompting techniques:
- Zero-shot/In-context
- Few-shot
- Chain-of-thoughts

In [ ]:
import pandas as pd
import re
import requests
import time
import os
import json
from tqdm import tqdm
import base64
import pathlib
import warnings
import http.client
import typing
import urllib.request
from PIL import Image, ImageOps, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import io
import numpy as np

from dotenv import load_dotenv
from groq import Groq
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai
import openai

warnings.filterwarnings("ignore")
from urllib3.exceptions import InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)


from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

In [ ]:
# Initialize environment variables and APIs
load_dotenv()

# Vertex AI initialization
project_id = os.getenv('VERTEXAI_PROJECT_ID')
vertexai.init(project=project_id, location="us-central1")
text_model = GenerativeModel("gemini-1.5-pro")

# OpenAI initialization
openai.api_key = os.getenv('OPENAI_API_KEY')

# Groq initialization
groq_api_key = os.getenv('GROQ_API_KEY')
groq_client = Groq(api_key=groq_api_key)

# Create output directories maintaining original structure
OUTPUT_DIR = pathlib.Path('Data')
GEMINI_DIR = OUTPUT_DIR / 'Gemini'
LLAMA_DIR = OUTPUT_DIR / 'llama-3.2'
GPT_DIR = OUTPUT_DIR / 'gpt-4'

# Create directories if they don't exist
for dir_path in [GEMINI_DIR, LLAMA_DIR, GPT_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

def create_similarity_directories():
    """Create directories for similarity analysis results"""
    for base_dir in [GEMINI_DIR, LLAMA_DIR, GPT_DIR]:
        similarity_dir = base_dir / 'similarity_analysis'
        similarity_dir.mkdir(parents=True, exist_ok=True)

# Call this after your existing directory creation
create_similarity_directories()

In [ ]:
# Utility functions
def encode_image_to_base64(image_url):
    """Convert image to base64 string"""
    response = requests.get(image_url, headers={'User-Agent': 'Mozilla/5.0'})
    response.raise_for_status()
    return base64.b64encode(response.content).decode('utf-8')

def resize_image(image, max_dimension):
    """Resize image maintaining aspect ratio"""
    width, height = image.size
    if image.mode == "P":
        if "transparency" in image.info:
            image = image.convert("RGBA")
        else:
            image = image.convert("RGB")
    if width > max_dimension or height > max_dimension:
        if width > height:
            new_width = max_dimension
            new_height = int(height * (max_dimension / width))
        else:
            new_height = max_dimension
            new_width = int(width * (max_dimension / height))
        image = image.resize((new_width, new_height), Image.LANCZOS)
    return image

def process_image(image_url, max_size):
    """Process image from URL"""
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image_data = io.BytesIO(response.content)

        with Image.open(image_data) as image:
            width, height = image.size
            mimetype = image.get_format_mimetype()

            if mimetype != "image/png" or width > max_size or height > max_size:
                resized_image = resize_image(image, max_size)
                png_data = io.BytesIO()
                resized_image.save(png_data, format="PNG")
                return base64.b64encode(png_data.getvalue()).decode('utf-8')
            else:
                return base64.b64encode(response.content).decode('utf-8')
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

def combine_images_vertical(images, max_size):
    """Combine multiple images into a vertical collage with labels"""
    processed_images = []
    for img_url in images:
        processed = process_image(img_url, max_size)
        if processed:
            processed_images.append(processed)

    if not processed_images:
        return None

    pil_images = []
    for img_data in processed_images:
        try:
            img_bytes = base64.b64decode(img_data)
            img = Image.open(io.BytesIO(img_bytes))
            pil_images.append(img)
        except Exception as e:
            print(f"Error converting base64 to image: {e}")
            continue

    if not pil_images:
        return None

    max_width = max(img.width for img in pil_images)
    total_height = sum(img.height for img in pil_images) + (len(pil_images) * 30)

    collage = Image.new("RGB", (max_width, total_height), "white")
    draw = ImageDraw.Draw(collage)

    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()

    y_offset = 0
    for i, img in enumerate(pil_images, 1):
        draw.text((10, y_offset), f"Image {i}", fill="black", font=font)
        y_offset += 30
        x_offset = (max_width - img.width) // 2
        collage.paste(img, (x_offset, y_offset))
        y_offset += img.height

    buffer = io.BytesIO()
    collage.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")


def create_embeddings_and_analyze(df, output_dir):
    """Create embeddings using MiniLM and analyze similarities"""
    # Initialize the model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    # Generate embeddings for original content
    print("Generating embeddings for original content...")
    title_embeddings = model.encode(df['Title'].tolist(), show_progress_bar=True)
    body_embeddings = model.encode(df['Body'].tolist(), show_progress_bar=True)

    # Generate embeddings for LLM responses
    print("\nGenerating embeddings for LLM responses...")
    response_embeddings = {
        'zero_shot_title': model.encode(df['llm_title_response'].tolist(), show_progress_bar=True),
        'zero_shot_body': model.encode(df['llm_body_response'].tolist(), show_progress_bar=True),
    }

    # Calculate similarities
    similarities = {
        'title': {},
        'body': {}
    }

    # Calculate similarities
    similarities['title']['zero_shot'] = np.diagonal(
        cosine_similarity(title_embeddings, response_embeddings['zero_shot_title'])
    )
    similarities['body']['zero_shot'] = np.diagonal(
        cosine_similarity(body_embeddings, response_embeddings['zero_shot_body'])
    )

    # Add similarity scores to dataframe
    df['similarity_title'] = similarities['title']['zero_shot']
    df['similarity_body'] = similarities['body']['zero_shot']

    # Save embeddings
    np.save(output_dir / 'similarity_analysis' / 'title_embeddings.npy', title_embeddings)
    np.save(output_dir / 'similarity_analysis' / 'body_embeddings.npy', body_embeddings)

    return df, similarities


def analyze_similarities(similarities, output_dir):
    """Analyze and visualize similarity distributions"""
    categories = {
        'Very High': (0.8, 1.0),
        'High': (0.6, 0.8),
        'Moderate': (0.4, 0.6),
        'Low': (0.2, 0.4),
        'Very Low': (0.0, 0.2)
    }

    results = {
        'title': {},
        'body': {}
    }

    # Create figures for each type
    for response_category in ['title', 'body']:
        fig, ax = plt.subplots(figsize=(10, 5))
        scores = similarities[response_category]['zero_shot']

        stats = {
            'mean': np.mean(scores),
            'median': np.median(scores),
            'std': np.std(scores),
            'min': np.min(scores),
            'max': np.max(scores)
        }

        distribution = {}
        for category, (low, high) in categories.items():
            count = np.sum((scores >= low) & (scores < high))
            percentage = (count / len(scores)) * 100
            distribution[category] = percentage

        results[response_category]['zero_shot'] = {
            'statistics': stats,
            'distribution': distribution
        }

        sns.histplot(scores, bins=30, ax=ax)
        ax.set_title(f'Similarity Distribution for {response_category.title()}')
        ax.set_xlabel('Similarity Score')
        ax.set_ylabel('Count')

        for category, (low, high) in categories.items():
            if low > 0:
                ax.axvline(x=low, color='r', linestyle='--', alpha=0.3)

        plt.tight_layout()
        plt.savefig(output_dir / 'similarity_analysis' / f'similarity_distribution_{response_category}.png')
        plt.close()

    return results

def process_with_similarity(model_name, df, output_dir):
    """Process results with similarity analysis"""
    # Create embeddings and analyze similarities
    print(f"\nAnalyzing similarities for {model_name}...")
    df_with_similarities, similarities = create_embeddings_and_analyze(df, output_dir)

    # Analyze and visualize results
    analysis_results = analyze_similarities(similarities, output_dir)

    # Save results
    output_file = output_dir / 'similarity_analysis' / f'responses_with_similarities.csv'
    df_with_similarities.to_csv(output_file, index=False)

    # Save analysis results
    with open(output_dir / 'similarity_analysis' / f'similarity_analysis_results.json', 'w') as f:
        json.dump(analysis_results, f, indent=4)

    return df_with_similarities, analysis_results

### Gemini Processing

In [ ]:
def gemini_generate_title(image_url, mode='zero-shot'):
    """Generate title using Gemini with different prompting techniques"""
    try:
        image = Image.from_bytes(requests.get(image_url).content)

        if mode == 'zero-shot':
            instruction = '''
Context: You are an expert programmer experienced in different technology stacks. You encountered an issue while working on a project. The screenshot shows the problem but you are not given any textual content.
Generate a Stack Overflow title that:
1. Follows Stack Overflow guidelines
2. Is clear and concise
3. Summarizes the main technical issue shown
4. Take inspiration from your trained data for stack overflow.

Output Format:
TITLE: <<Generated Title>>

Make sure to start with exactly "TITLE:"
'''
        elif mode == 'chain-of-thoughts':
            instruction = '''
Reasoning Process for Title Generation:

1. Initial Observation
- What is immediately visible in the screenshot?
- What IDE/tool is being used or what kind of code is shown?
- Are there any error messages or unusual indicators?

2. Problem Identification
- What seems to be the main issue?
- Which specific components are involved?
- Is this a configuration, syntax, or runtime issue?

3. Title Formulation
Based on the above analysis, construct a title that:
- Clearly summarizes the main technical issue
- Uses relevant technical keywords
- Is concise and specific
- Takes inspiration from your trained data for stack overflow
- Would be easily searchable

Output Format:
TITLE: <<Generated Title>>
Make sure to start with exactly "TITLE:"
'''
        else:  # few-shot
            example_image1 = Image.from_bytes(requests.get("https://i.sstatic.net/rUHWv1Ok.png").content)
            example_text1 = "TITLE: Trying to Stack 2 Columns into one Excel"
            example_image2 = Image.from_bytes(requests.get("https://i.sstatic.net/TGFPo9Jj.png").content)
            example_text2 = "TITLE: is getenv_s not part of cstdlib?"

            instruction = '''
You are an expert software developer and Stack Overflow analyst. You are given the following examples:
'''
            prompt = '''
Now generate a similar title for the given screenshot:
Follow the pattern:
1. Clear and concise title that summarizes the main issue
2. Take inspiration from the example titles given

Output Format:
TITLE: <<Generated Title>>
Make sure to start with exactly "TITLE:"
'''
            contents = [instruction, example_image1, example_text1, example_image2, example_text2, image, prompt]

        if mode != 'few-shot':
            contents = [instruction, image]

        generation_config = GenerationConfig(
            temperature=0,
            top_p=0.8,
            top_k=40,
            candidate_count=1,
            max_output_tokens=2048,
        )

        response = text_model.generate_content(
            contents,
            generation_config=generation_config,
            stream=True,
        )

        result = ''
        for r in response:
            result += r.text

        if "TITLE:" in result:
            return result.split("TITLE:")[1].strip()
        return result

    except Exception as e:
        print(f"Error processing with Gemini: {e}")
        return ""

def gemini_generate_body(image_url, mode='zero-shot'):
    """Generate body using Gemini with different prompting techniques"""
    try:
        image = Image.from_bytes(requests.get(image_url).content)

        if mode == 'zero-shot':
            instruction = '''
Context: You are an expert programmer experienced in different technology stacks. You encountered an issue while working on a project. The screenshot shows the problem but you are not given any textual content.
Generate a detailed Stack Overflow question body that:
1. Follows Stack Overflow guidelines
2. Includes relevant code/IDE context
3. Clearly states the expected vs. actual behavior
4. Take inspiration from your trained data for stack overflow.

Output Format:
BODY: <<Generated Body>>

Make sure to start with exactly "BODY:"
'''
        elif mode == 'chain-of-thoughts':
            instruction = '''
Reasoning Process for Body Generation:

1. Initial Observation
- What is immediately visible in the screenshot?
- What IDE/tool is being used or what kind of code is shown?
- Are there any error messages or unusual indicators?

2. Problem Identification
- What seems to be the main issue?
- Which specific components are involved?
- Is this a configuration, syntax, or runtime issue?

3. Context Building
- What background or programming information is needed to understand this issue?
- Which framework/language versions are relevant?
- What might have led to this situation?

4. Solution Attempts Analysis
- What obvious solutions might have been tried?
- What documentation might be relevant?
- What troubleshooting steps would make sense?

5. Question Body Formulation
Based on the above analysis, construct a detailed body that:
- Clearly explains the context and problem
- Includes all relevant technical details
- Takes inspiration from your trained data for stack overflow
- Shows research effort and attempted solutions
- Is specific and answerable

Output Format:
BODY: <<Generated Body>>
Make sure to start with exactly "BODY:"
'''
        else:  # few-shot
            example_image1 = Image.from_bytes(requests.get("https://i.sstatic.net/rUHWv1Ok.png").content)
            example_text1 = '''
BODY: I am currently attempting to combine two columns into one but have encountered an error that prevents me from completing this task. Additionally, the data from the second column appears to be pasting incorrectly after the error messages.I would greatly appreciate any assistance with this issue.=IF(P2<>"",P2,INDEX($R$2:$R$5000,ROW()-COUNTA($P$2:$P$5000)))Column P and R contains formulas.
'''
            example_image2 = Image.from_bytes(requests.get("https://i.sstatic.net/TGFPo9Jj.png").content)
            example_text2 = '''
BODY: C11 added new bounds-checked functions to the standard library, such as getenv_s.However, when I include <cstdlib>, I do not have std::getenv_s, only getenv_s (global namespace).cppreference has the following note:As with all bounds-checked functions, getenv_s is only guaranteed to be available if __STDC_LIB_EXT1__ is defined by the implementation and if the user defines __STDC_WANT_LIB_EXT1__ to the integer constant 1 before including <stdlib.h>.Even when I define __STDC_WANT_LIB_EXT1__ as 1, My compiler (MSVC C++23) does not find the std::getenv_s function.Isn't <cstdlib> supposed to bring every symbol of <stdlib.h> into the std namespace?
'''
            instruction = '''
You are an expert software developer and Stack Overflow analyst. You are given the following examples:
'''
            prompt = '''
Now generate a similar question body for the given screenshot:
Follow the pattern:
1. Clear and concise question body that explains the issue in detail
2. What is the most important part of an error that can be present in the image?
3. Take inspiration from the example bodies given

Output Format:
BODY: <<Generated Body>>
Make sure to start with exactly "BODY:"
'''
            contents = [instruction, example_image1, example_text1, example_image2, example_text2, image, prompt]

        if mode != 'few-shot':
            contents = [instruction, image]

        generation_config = GenerationConfig(
            temperature=0,
            top_p=0.8,
            top_k=40,
            candidate_count=1,
            max_output_tokens=2048,
        )

        response = text_model.generate_content(
            contents,
            generation_config=generation_config,
            stream=True,
        )

        result = ''
        for r in response:
            result += r.text

        if "BODY:" in result:
            return result.split("BODY:")[1].strip()
        return result

    except Exception as e:
        print(f"Error processing with Gemini: {e}")
        return ""

def process_with_gemini(dataframe, mode='zero-shot'):
    """Process entire dataset using Gemini"""
    results = []

    # Create log file
    log_file = GEMINI_DIR / f'processing_log_{mode}.txt'
    with open(log_file, 'w') as f:
        f.write(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")

    for i, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0], desc='Processing'):
        print(f"\nProcessing Row ID: {row['Id']}")

        image_urls = [url.strip(",") for url in re.findall(r"'([^']*)'", str(row.get('ImageURLs')))]
        title_responses = []
        body_responses = []

        for image_url in image_urls:
            title_text = gemini_generate_title(image_url, mode)
            body_text = gemini_generate_body(image_url, mode)

            title_responses.append(title_text)
            body_responses.append(body_text)

            with open(log_file, 'a') as f:
                f.write(f"\nProcessed Row ID: {row['Id']} at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Title response: {title_text}\n")
                f.write(f"Body response: {body_text}\n")

        results.append({
            'Id': row['Id'],
            'Title': row['Title'],
            'Body': row['Body'],
            'ImageURLs': row['ImageURLs'],
            'llm_title_response': ' ||| '.join(title_responses),
            'llm_body_response': ' ||| '.join(body_responses)
        })

        if len(results) % 2 == 0:
            # Save intermediate results
            intermediate_df = pd.DataFrame(results)
            intermediate_file = GEMINI_DIR / f'intermediate_results_{mode}_{time.strftime("%Y%m%d_%H%M%S")}.csv'
            intermediate_df.to_csv(intermediate_file, index=False)
            print(f"Saved intermediate results to {intermediate_file}")

    # Save final results
    final_df = pd.DataFrame(results)
    final_file = GEMINI_DIR / f'llm_responses_final_{mode}.csv'
    final_df.to_csv(final_file, index=False)

    final_df_with_similarities, similarity_results = process_with_similarity(
        'gemini',
        final_df,
        GEMINI_DIR
    )

    return final_df_with_similarities

### Llama Processing

In [ ]:
def llama_generate_title(image_url, mode='zero-shot'):
    """Generate title using LLaMA with different prompting techniques"""
    try:
        base64_image = encode_image_to_base64(image_url)

        if mode == 'zero-shot':
            instruction = '''
Context: You are an expert programmer experienced in different technology stacks. You encountered an issue while working on a project. The screenshot shows the problem but you are not given any textual content.
Generate a Stack Overflow title that:
1. Follows Stack Overflow guidelines
2. Is clear and concise
3. Summarizes the main technical issue shown
4. Take inspiration from your trained data for stack overflow.

Output Format:
TITLE: <<Generated Title>>

Make sure to start with exactly "TITLE:"
'''
        elif mode == 'chain-of-thoughts':
            instruction = '''
Reasoning Process for Title Generation:

1. Initial Observation
- What is immediately visible in the screenshot?
- What IDE/tool is being used or what kind of code is shown?
- Are there any error messages or unusual indicators?

2. Problem Identification
- What seems to be the main issue?
- Which specific components are involved?
- Is this a configuration, syntax, or runtime issue?

3. Title Formulation
Based on the above analysis, construct a title that:
- Clearly summarizes the main technical issue
- Uses relevant technical keywords
- Is concise and specific
- Takes inspiration from your trained data for stack overflow
- Would be easily searchable

Output Format:
TITLE: <<Generated Title>>
Make sure to start with exactly "TITLE:"
'''
        else:  # few-shot
            instruction = '''
You are an expert software developer and Stack Overflow analyst. You are given the following examples:

Example 1:
Title: "Trying to Stack 2 Columns into one Excel"

Example 2:
Title: "is getenv_s not part of cstdlib?"

Now generate a similar title for the given screenshot:
Follow the pattern:
1. Clear and concise title that summarizes the main issue
2. Take inspiration from the example titles given

Output Format:
TITLE: <<Generated Title>>
Make sure to start with exactly "TITLE:"
'''

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": instruction},
                    {"type": "image_url",
                     "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ]

        response = groq_client.chat.completions.create(
            messages=messages,
            model="llama-3.2-90b-vision-preview",
            temperature=0,
            max_tokens=2048
        )

        result = response.choices[0].message.content
        if "TITLE:" in result:
            return result.split("TITLE:")[1].strip()
        return result

    except Exception as e:
        print(f"Error processing with LLaMA: {e}")
        return ""

def llama_generate_body(image_url, mode='zero-shot'):
    """Generate body using LLaMA with different prompting techniques"""
    try:
        base64_image = encode_image_to_base64(image_url)

        if mode == 'zero-shot':
            instruction = '''
Context: You are an expert programmer experienced in different technology stacks. You encountered an issue while working on a project. The screenshot shows the problem but you are not given any textual content.
Generate a detailed Stack Overflow question body that:
1. Follows Stack Overflow guidelines
2. Includes relevant code/IDE context
3. Clearly states the expected vs. actual behavior
4. Take inspiration from your trained data for stack overflow.

Output Format:
BODY: <<Generated Body>>

Make sure to start with exactly "BODY:"
'''
        elif mode == 'chain-of-thoughts':
            instruction = '''
Reasoning Process for Body Generation:

1. Initial Observation
- What is immediately visible in the screenshot?
- What IDE/tool is being used or what kind of code is shown?
- Are there any error messages or unusual indicators?

2. Problem Identification
- What seems to be the main issue?
- Which specific components are involved?
- Is this a configuration, syntax, or runtime issue?

3. Context Building
- What background or programming information is needed to understand this issue?
- Which framework/language versions are relevant?
- What might have led to this situation?

4. Solution Attempts Analysis
- What obvious solutions might have been tried?
- What documentation might be relevant?
- What troubleshooting steps would make sense?

5. Question Body Formulation
Based on the above analysis, construct a detailed body that:
- Clearly explains the context and problem
- Includes all relevant technical details
- Takes inspiration from your trained data for stack overflow
- Shows research effort and attempted solutions
- Is specific and answerable

Output Format:
BODY: <<Generated Body>>
Make sure to start with exactly "BODY:"
'''
        else:  # few-shot
            instruction = '''
You are an expert software developer and Stack Overflow analyst. You are given the following examples:

Example 1:
Body: "I am currently attempting to combine two columns into one but have encountered an error that prevents me from completing this task. Additionally, the data from the second column appears to be pasting incorrectly after the error messages.I would greatly appreciate any assistance with this issue.=IF(P2<>"",P2,INDEX($R$2:$R$5000,ROW()-COUNTA($P$2:$P$5000)))Column P and R contains formulas."

Example 2:
Body: "C11 added new bounds-checked functions to the standard library, such as getenv_s. However, when I include <cstdlib>, I do not have std::getenv_s, only getenv_s (global namespace). cppreference has the following note: As with all bounds-checked functions, getenv_s is only guaranteed to be available if __STDC_LIB_EXT1__ is defined by the implementation and if the user defines __STDC_WANT_LIB_EXT1__ to the integer constant 1 before including <stdlib.h>. Even when I define __STDC_WANT_LIB_EXT1__ as 1, My compiler (MSVC C++23) does not find the std::getenv_s function. Isn't <cstdlib> supposed to bring every symbol of <stdlib.h> into the std namespace?"

Now generate a similar question body for the given screenshot:
Follow the pattern:
1. Clear and concise question body that explains the issue in detail
2. What is the most important part of an error that can be present in the image?
3. Take inspiration from the example bodies given

Output Format:
BODY: <<Generated Body>>
Make sure to start with exactly "BODY:"
'''

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": instruction},
                    {"type": "image_url",
                     "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ]

        response = groq_client.chat.completions.create(
            messages=messages,
            model="llama-3.2-90b-vision-preview",
            temperature=0,
            max_tokens=4096
        )

        result = response.choices[0].message.content
        if "BODY:" in result:
            return result.split("BODY:")[1].strip()
        return result

    except Exception as e:
        print(f"Error processing with LLaMA: {e}")
        return ""

def process_with_llama(dataframe, mode='zero-shot'):
    """Process entire dataset using LLaMA"""
    results = []

    # Create log file
    log_file = LLAMA_DIR / f'processing_log_{mode}.txt'
    with open(log_file, 'w') as f:
        f.write(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")

    for i, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0], desc='Processing'):
        print(f"\nProcessing Row ID: {row['Id']}")

        image_urls = [url.strip(",") for url in re.findall(r"'([^']*)'", str(row.get('ImageURLs')))]
        title_responses = []
        body_responses = []

        for image_url in image_urls:
            title_text = llama_generate_title(image_url, mode)
            body_text = llama_generate_body(image_url, mode)

            title_responses.append(title_text)
            body_responses.append(body_text)

            with open(log_file, 'a') as f:
                f.write(f"\nProcessed Row ID: {row['Id']} at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Title response: {title_text}\n")
                f.write(f"Body response: {body_text}\n")

        results.append({
            'Id': row['Id'],
            'Title': row['Title'],
            'Body': row['Body'],
            'ImageURLs': row['ImageURLs'],
            'llm_title_response': ' ||| '.join(title_responses),
            'llm_body_response': ' ||| '.join(body_responses)
        })

        if len(results) % 2 == 0:
            # Save intermediate results
            intermediate_df = pd.DataFrame(results)
            intermediate_file = LLAMA_DIR / f'intermediate_results_{mode}_{time.strftime("%Y%m%d_%H%M%S")}.csv'
            intermediate_df.to_csv(intermediate_file, index=False)
            print(f"Saved intermediate results to {intermediate_file}")

    # Save final results
    final_df = pd.DataFrame(results)
    final_file = LLAMA_DIR / f'llm_responses_final_{mode}.csv'
    final_df.to_csv(final_file, index=False)

    final_df_with_similarities, similarity_results = process_with_similarity(
        'llama',
        final_df,
        LLAMA_DIR
    )

    return final_df_with_similarities

### GPT Processing

In [ ]:
def gpt_generate_title(image_url, mode='zero-shot'):
    """Generate title using GPT-4 Vision with different prompting techniques"""
    try:
        base64_image = encode_image_to_base64(image_url)

        if mode == 'zero-shot':
            instruction = '''
Context: You are an expert programmer experienced in different technology stacks. You encountered an issue while working on a project. The screenshot shows the problem but you are not given any textual content.
Generate a Stack Overflow title that:
1. Follows Stack Overflow guidelines
2. Is clear and concise
3. Summarizes the main technical issue shown
4. Take inspiration from your trained data for stack overflow.

Output Format:
TITLE: <<Generated Title>>

Make sure to start with exactly "TITLE:"
'''
        elif mode == 'chain-of-thoughts':
            instruction = '''
Let's think about this step by step:

1. Initial Visual Analysis
- What are we seeing in the screenshot?
- What IDE/tool/interface is being used?
- Are there any error messages or warnings?

2. Technical Context
- What programming language/framework is involved?
- What specific components or features are we working with?
- Is there any configuration or setup visible?

3. Problem Pattern Recognition
- What type of issue is this (syntax error, runtime error, unexpected behavior)?
- What specific error messages or unexpected outputs are shown?
- What seems to be the root cause based on the visual evidence?

4. Title Construction
Now, based on this analysis, construct a title that:
- Clearly identifies the main technical issue
- Includes relevant technical keywords
- Is concise yet informative
- Follows Stack Overflow best practices

Output Format:
TITLE: <<Generated Title>>
Make sure to start with exactly "TITLE:"
'''
        else:  # few-shot
            instruction = '''
You are an expert software developer and Stack Overflow analyst. Here are two examples of effective Stack Overflow titles for reference:

Example 1: "Trying to Stack 2 Columns into one Excel"
- Notice how it clearly states the action and the specific Excel operation

Example 2: "is getenv_s not part of cstdlib?"
- Observe how it specifically names the function and library in question

Now, looking at the provided screenshot:
1. Follow a similar pattern of clarity and specificity
2. Focus on the main technical issue shown
3. Use relevant technical terms from the image

Output Format:
TITLE: <<Generated Title>>
Make sure to start with exactly "TITLE:"
'''

        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": instruction
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ]

        response = openai.ChatCompletion.create(
            model="gpt4o",
            messages=messages,
            max_tokens=2048,
            temperature=0
        )

        result = response.choices[0].message.content
        if "TITLE:" in result:
            return result.split("TITLE:")[1].strip()
        return result

    except Exception as e:
        print(f"Error processing with GPT: {e}")
        return ""

def gpt_generate_body(image_url, mode='zero-shot'):
    """Generate body using GPT-4 Vision with different prompting techniques"""
    try:
        base64_image = encode_image_to_base64(image_url)

        if mode == 'zero-shot':
            instruction = '''
Context: You are an expert programmer experienced in different technology stacks. You encountered an issue while working on a project. The screenshot shows the problem but you are not given any textual content.
Generate a detailed Stack Overflow question body that:
1. Follows Stack Overflow guidelines
2. Includes relevant code/IDE context
3. Clearly states the expected vs. actual behavior
4. Take inspiration from your trained data for stack overflow.

Output Format:
BODY: <<Generated Body>>

Make sure to start with exactly "BODY:"
'''
        elif mode == 'chain-of-thoughts':
            instruction = '''
Let's analyze this systematically:

1. Visual Analysis
- What do we see in the screenshot?
- What development environment or tool is shown?
- What UI elements or code snippets are visible?

2. Technical Context Identification
- Which technologies/frameworks are involved?
- What version or environment details are relevant?
- What setup or configuration is implied?

3. Problem Description Formation
- What is the main issue or error shown?
- What were the expected and actual outcomes?
- What relevant code or configuration is visible?

4. Attempt Documentation
- What approaches might have already been tried?
- What common solutions would need to be ruled out?
- What troubleshooting steps are implied?

5. Question Formulation
Based on this analysis, create a body that:
- Clearly describes the technical context
- Documents the specific issue
- Shows research effort
- Includes relevant code/configuration
- Asks a specific, answerable question

Output Format:
BODY: <<Generated Body>>
Make sure to start with exactly "BODY:"
'''
        else:  # few-shot
            instruction = '''
You are an expert software developer and Stack Overflow analyst. Here are two examples of effective question bodies:

Example 1:
"I am currently attempting to combine two columns into one but have encountered an error that prevents me from completing this task. Additionally, the data from the second column appears to be pasting incorrectly after the error messages. I would greatly appreciate any assistance with this issue. =IF(P2<>"",P2,INDEX($R$2:$R$5000,ROW()-COUNTA($P$2:$P$5000))) Column P and R contains formulas."

Example 2:
"C11 added new bounds-checked functions to the standard library, such as getenv_s. However, when I include <cstdlib>, I do not have std::getenv_s, only getenv_s (global namespace). cppreference has the following note: As with all bounds-checked functions, getenv_s is only guaranteed to be available if __STDC_LIB_EXT1__ is defined by the implementation and if the user defines __STDC_WANT_LIB_EXT1__ to the integer constant 1 before including <stdlib.h>. Even when I define __STDC_WANT_LIB_EXT1__ as 1, My compiler (MSVC C++23) does not find the std::getenv_s function. Isn't <cstdlib> supposed to bring every symbol of <stdlib.h> into the std namespace?"

Now, create a similar detailed body for the issue shown in the screenshot:
1. Follow the pattern of clear problem description
2. Include relevant technical details
3. Show attempted solutions if visible
4. Ask clear, specific questions

Output Format:
BODY: <<Generated Body>>
Make sure to start with exactly "BODY:"
'''

        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": instruction
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ]

        response = openai.ChatCompletion.create(
            model="gpt4o",
            messages=messages,
            max_tokens=4096,
            temperature=0
        )

        result = response.choices[0].message.content
        if "BODY:" in result:
            return result.split("BODY:")[1].strip()
        return result

    except Exception as e:
        print(f"Error processing with GPT: {e}")
        return ""

def process_with_gpt(dataframe, mode='zero-shot'):
    """Process entire dataset using GPT-4 Vision"""
    results = []

    # Create log file
    log_file = GPT_DIR / f'processing_log_{mode}.txt'
    with open(log_file, 'w') as f:
        f.write(f"Processing started at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")

    for i, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0], desc='Processing'):
        print(f"\nProcessing Row ID: {row['Id']}")

        image_urls = [url.strip(",") for url in re.findall(r"'([^']*)'", str(row.get('ImageURLs')))]
        title_responses = []
        body_responses = []

        for image_url in image_urls:
            title_text = gpt_generate_title(image_url, mode)
            body_text = gpt_generate_body(image_url, mode)

            title_responses.append(title_text)
            body_responses.append(body_text)

            with open(log_file, 'a') as f:
                f.write(f"\nProcessed Row ID: {row['Id']} at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Title response: {title_text}\n")
                f.write(f"Body response: {body_text}\n")

        results.append({
            'Id': row['Id'],
            'Title': row['Title'],
            'Body': row['Body'],
            'ImageURLs': row['ImageURLs'],
            'llm_title_response': ' ||| '.join(title_responses),
            'llm_body_response': ' ||| '.join(body_responses)
        })

        if len(results) % 2 == 0:
            # Save intermediate results
            intermediate_df = pd.DataFrame(results)
            intermediate_file = GPT_DIR / f'intermediate_results_{mode}_{time.strftime("%Y%m%d_%H%M%S")}.csv'
            intermediate_df.to_csv(intermediate_file, index=False)
            print(f"Saved intermediate results to {intermediate_file}")

    # Save final results
    final_df = pd.DataFrame(results)
    final_file = GPT_DIR / f'llm_responses_final_{mode}.csv'
    final_df.to_csv(final_file, index=False)

    final_df_with_similarities, similarity_results = process_with_similarity(
        'gpt',
        final_df,
        GPT_DIR
    )

    return final_df_with_similarities

In [ ]:
# Cell 7: Load Dataset and Main Processing Pipeline
# Load dataset
dataset = pd.read_csv('Data/filtered_data_matched.csv')
print(f"Loaded dataset with {len(dataset)} rows")

def process_all_models():
    """Process dataset with all models and prompting techniques, including similarity analysis"""

    # Dictionary to store all results
    all_results = {
        'gemini': {},
        'llama': {},
        'gpt': {}
    }

    prompting_techniques = ['zero-shot', 'few-shot', 'chain-of-thoughts']

    for technique in prompting_techniques:
        print(f"\n{'='*50}")
        print(f"Processing with {technique} prompting")
        print(f"{'='*50}")

        # Process with Gemini
        print("\nStarting Gemini processing...")
        gemini_results = process_with_gemini(dataset, mode=technique)
        all_results['gemini'][technique] = {
            'data': gemini_results,
            'dir': GEMINI_DIR
        }
        print(f"Completed Gemini processing with similarity analysis")

        # Process with LLaMA
        print("\nStarting LLaMA processing...")
        llama_results = process_with_llama(dataset, mode=technique)
        all_results['llama'][technique] = {
            'data': llama_results,
            'dir': LLAMA_DIR
        }
        print(f"Completed LLaMA processing with similarity analysis")

        # Process with GPT
        print("\nStarting GPT processing...")
        gpt_results = process_with_gpt(dataset, mode=technique)
        all_results['gpt'][technique] = {
            'data': gpt_results,
            'dir': GPT_DIR
        }
        print(f"Completed GPT processing with similarity analysis")

        print(f"\nCompleted {technique} processing for all models")

        # Generate comparative analysis
        compare_model_similarities(
            technique,
            gemini_results,
            llama_results,
            gpt_results,
            OUTPUT_DIR / f'comparative_analysis_{technique}'
        )

    return all_results

def compare_model_similarities(technique, gemini_df, llama_df, gpt_df, output_dir):
    """Compare similarity scores across models"""
    output_dir = pathlib.Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Prepare comparison data
    comparison_data = {
        'gemini': {
            'title': gemini_df['similarity_title'],
            'body': gemini_df['similarity_body']
        },
        'llama': {
            'title': llama_df['similarity_title'],
            'body': llama_df['similarity_body']
        },
        'gpt': {
            'title': gpt_df['similarity_title'],
            'body': gpt_df['similarity_body']
        }
    }

    # Create comparison plots
    for content_type in ['title', 'body']:
        plt.figure(figsize=(12, 6))
        for model in ['gemini', 'llama', 'gpt']:
            sns.kdeplot(
                data=comparison_data[model][content_type],
                label=model.upper()
            )

        plt.title(f'Similarity Score Distribution Comparison - {content_type.title()} ({technique})')
        plt.xlabel('Similarity Score')
        plt.ylabel('Density')
        plt.legend()
        plt.savefig(output_dir / f'similarity_comparison_{content_type}.png')
        plt.close()

    # Calculate summary statistics
    summary_stats = {
        'technique': technique,
        'models': {}
    }

    for model in ['gemini', 'llama', 'gpt']:
        summary_stats['models'][model] = {
            'title': {
                'mean': np.mean(comparison_data[model]['title']),
                'median': np.median(comparison_data[model]['title']),
                'std': np.std(comparison_data[model]['title'])
            },
            'body': {
                'mean': np.mean(comparison_data[model]['body']),
                'median': np.median(comparison_data[model]['body']),
                'std': np.std(comparison_data[model]['body'])
            }
        }

    # Save summary statistics
    with open(output_dir / 'comparison_summary.json', 'w') as f:
        json.dump(summary_stats, f, indent=4)

if __name__ == "__main__":
    results = process_all_models()
    print("\nProcessing completed for all models with similarity analysis")